## Illustrative CNN Model
Goal: Define which model architecture is best suited for synthetic data training and testing. 

In [2]:
import tensorflow as tf

#keras is high level application program interface
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import time
import os
import datetime
import numpy as np

import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import TensorBoard #interface with tensorboard using keras callback

import pickle


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

# Access tensorboard GPU

In [5]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=59641c3638b655ea433b5584b623e22e17352b972eb9654aa830e188be383ac7
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.3 GB  | Proc size: 859.1 MB
GPU RAM Free: 11307MB | Used: 134MB | Util   1% | Total 11441MB


In [6]:
pwd

'/content'

In [7]:
tf.keras.backend.clear_session()

#Set up model paramaters

In [ ]:

#log_dir="\\content\\CNN_7_13\\logs\\{}"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir="/content/drive/My Drive/CNN_7_16/All Python/logs/{}"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


#/content/drive/My Drive/CNN_7_16/All Python/logs

X = np.array(pickle.load(open("/content/drive/My Drive/CNN_7_16/All Python/X1.pickle","rb"))) #images
y = np.array(pickle.load(open("/content/drive/My Drive/CNN_7_16/All Python/y1.pickle","rb"))) #labels

X = X/255.0 #max number of pixels, normalizing data, scale imagery

#can use modelcheckpoint to save on best loss, val accuracy, etc., 
dense_layers= [0] # [0,1,2]
layer_sizes = [64]  # [32,64,128]
conv_layers=  [3] # [1,2,3]


for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME= "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer,layer_size, dense_layer, int(time.time())) #begin to iterate through models
            tensorboard = TensorBoard(log_dir='/content/drive/My Drive/CNN_7_16/All Python/logs/{}'.format(NAME))     #=log_dir,histogram_freq=1)
            print(NAME)
            
            model = Sequential()

#layer 1

# pass in number of neurons per layer, then window size we want, then in input shape skip how many feature sets we have
# for input shape, skip -1 (how many feature sets we had), start at 1. 

#I can't figure out why when I start at 1 with color it throws off the # of input samples vs target samples
#when it seems like it should start at 300 vs 238.. 
            model.add(Conv2D(layer_size,(3,3), input_shape=X.shape[1:])) 
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
            #layer 2
                
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))          
                
            #layer 3   
            
            #flatten data
            model.add(Flatten()) #Convert 3D feature maps into 1D feature vectors
            
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))
            
            #output layer 
            
            model.add(Dense(1)) #Fully connected
            model.add(Activation("sigmoid"))
            
            model.compile(loss="binary_crossentropy",
                         optimizer = "adam",
                         metrics = ['accuracy']) #could use categorical versus bin cross, adam adaptive learning rate
            
            
            #batch size smaller may be better, epochs see where converge, val_split is a percentage chunk to use for validation testing
            
            t0=time.time()
            history = model.fit(X,y, batch_size=1, epochs = 20, validation_split=0.2, callbacks = [tensorboard])

In [10]:
print(y.shape)

(1000,)


In [11]:
print(NAME)

3-conv-64-nodes-0-dense-1596776053


In [20]:
#lots we can change to optomize the model 
#how can we use the model

#load in a model and test it on a new image

model.save('64x3-CNN.model')
#model.save('3-conv-64-nodes-0-dense-1596776053')
print(model)

INFO:tensorflow:Assets written to: 64x3-CNN.model/assets


In [21]:
import cv2
CATEGORIES = ["Neg_Training_Data","Pos_Training_Data"]

def prepare(filepath):
  IMG_SIZE=250
  img_array = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
  new_array=cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
  return new_array.reshape(-1,IMG_SIZE,IMG_SIZE,1)

model = tf.keras.models.load_model('64x3-CNN.model')

prediction = model.predict([prepare('/content/drive/My Drive/CNN_7_16/All Python/Synthetic_Data/practice/ExPlot7.png')]) #predict a list
print(CATEGORIES[int(prediction[0][0])])


Neg_Training_Data
